# Using spacy for POS tagging

In [1]:
import spacy
from spacy.lang.en.examples import sentences 

nlp = spacy.load("en_core_web_sm")
doc = nlp(sentences[0])
print("The text is: ", doc.text)
print("\n")
print("The part of speexh tagging for each text particle used is: ")
print ("{:<10} {:<15} {:<10}".format('Text','Part of speech','POS Dependency'))
print(" ")
for token in doc:
    print("{:<10} {:<15} {:<10}".format(token.text, token.pos_, token.dep_))

The text is:  Apple is looking at buying U.K. startup for $1 billion


The part of speexh tagging for each text particle used is: 
Text       Part of speech  POS Dependency
 
Apple      PROPN           nsubj     
is         AUX             aux       
looking    VERB            ROOT      
at         ADP             prep      
buying     VERB            pcomp     
U.K.       PROPN           compound  
startup    NOUN            dobj      
for        ADP             prep      
$          SYM             quantmod  
1          NUM             compound  
billion    NUM             pobj      


# Vectorizing and Doing a sentiment analysis on a random Data.

Step 1: Loading the dataset

In [1]:
import pandas as pd
df= pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
df['review'][0]  #First review of the data

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

Step 2: Data Preparation

In [3]:
# Use regex to scrape of noisy data.

Step 3: Stemming

In [4]:
from nltk.stem.porter import PorterStemmer
porter= PorterStemmer()

In [5]:
def stemmer_tokenize(text):
    return[porter.stem(word) for word in text.split()]

In [6]:
#To get rid of stop words use nltk.download('stopwords') which is not required here as the data is quite free.

In [7]:
stemmer_tokenize("Hello, may everyone feel free to be seated and get aboard for the upcoming journey!:)")

['hello,',
 'may',
 'everyon',
 'feel',
 'free',
 'to',
 'be',
 'seat',
 'and',
 'get',
 'aboard',
 'for',
 'the',
 'upcom',
 'journey!:)']

Step 4: TF-IDF Vectorization

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDV
tfidf= TFIDV(strip_accents= None,
            lowercase= False,
            tokenizer= stemmer_tokenize,
            use_idf= True,
            norm='l2',
            smooth_idf= True)
X= tfidf.fit_transform(df.review)
Y= df.sentiment.values

Step 5: Splitting the data into train and test

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(X,Y,random_state=1, test_size=0.5)

Step 6: Creating a testing model

In [13]:
import pickle
from sklearn.linear_model import LogisticRegressionCV as LRCV
clf= LRCV(cv=5,
         scoring= 'accuracy',
         random_state= 0,
         n_jobs= 1,
         verbose= 2,
         max_iter= 300).fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 11.5min finished


Step 7: Save the model in the present state using pickle

In [14]:
saved_model= open('saved_model.sav','wb')
pickle.dump(clf, saved_model)
saved_model.close()

Step 8:From the saved model analyse it's accuracy for the data

In [15]:
saved_clf= pickle.load(open('saved_model.sav','rb'))
accuracy= saved_clf.score(x_test, y_test)
accuracp=accuracy*100
print("The accuracy score is {acc:.2f}".format(acc=accuracp))

The accuracy score is 89.00
